In [ ]:
import sys
import os
import findspark
findspark.init()
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
import spark
import pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.functions import *

In [ ]:
spark = SparkSession.builder.appName('capstone_project').getOrCreate()

In [ ]:
user = os.getenv("user", default=None)
password = os.getenv("password", default=None)

In [ ]:
# creating creditcard datafram from the json file
cdw_sapp_credit = os.path.join("files", "cdw_sapp_credit.json")
df_creditcard = spark.read.json(cdw_sapp_credit)

In [ ]:
type(df_creditcard)
df_creditcard.show()

In [ ]:
df_creditcard.printSchema()

In [ ]:
df_creditcard.createOrReplaceTempView('credit_data')

In [ ]:
creditcard_df = spark.sql('SELECT CREDIT_CARD_NO CUST_CC_NO,\
         CONCAT(YEAR, LPAD(Month, 2, 0), \
         LPAD(Day, 2, 0)) TIMEID, \
         CUST_SSN,\
         BRANCH_CODE,\
         TRANSACTION_TYPE,\
         TRANSACTION_VALUE,\
         TRANSACTION_ID \
 FROM credit_data')
creditcard_df.show()


In [ ]:
creditcard_df.createOrReplaceTempView('credit_df')

In [ ]:
creditcard_df.printSchema()

In [ ]:
creditcard_df = spark.sql("SELECT CUST_CC_NO, TIMEID, CAST(CUST_SSN AS INT) CUST_SSN, \
CAST(BRANCH_CODE AS INT) BRANCH_CODE, TRANSACTION_TYPE, CAST(TRANSACTION_VALUE AS DOUBLE) TRANSACTION_VALUE, \
CAST(TRANSACTION_ID AS INT) TRANSACTION_ID FROM credit_df")

In [ ]:
creditcard_df.printSchema()

In [ ]:
creditcard_df.show()

In [ ]:
# ' DROP TABLE `creditcard_capstone`.`cdw_sapp_credit_card`;'

In [ ]:
creditcard_df.write.format("jdbc") \
.mode("append") \
.option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
.option("dbtable", "creditcard_capstone.cdw_sapp_credit_card") \
.option("user", user) \
.option("password", password) \
.save()

In [ ]:
spark.stop()

In [ ]:
# ALTER TABLE `creditcard_capstone`.`cdw_sapp_credit_card` 
# CHANGE COLUMN `CUST_CC_NO` `CUST_CC_NO` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `TIMEID` `TIMEID` VARCHAR(45) NULL DEFAULT NULL ,
# CHANGE COLUMN `TRANSACTION_TYPE` `TRANSACTION_TYPE` VARCHAR(45) NULL DEFAULT NULL ;

In [ ]:
# ALTER TABLE `creditcard_capstone`.`cdw_sapp_credit_card` 
# CHANGE COLUMN `CUST_CC_NO` `CUST_CC_NO` VARCHAR(45) NOT NULL ,
# ADD PRIMARY KEY (`CUST_CC_NO`),
# ADD INDEX `BRANCH_CODE_idx` (`BRANCH_CODE` ASC) VISIBLE,
# ADD INDEX `CUST_SSN_idx` (`CUST_SSN` ASC) VISIBLE;
# ;
# ALTER TABLE `creditcard_capstone`.`cdw_sapp_credit_card` 
# ADD CONSTRAINT `BRANCH_CODE`
#   FOREIGN KEY (`BRANCH_CODE`)
#   REFERENCES `creditcard_capstone`.`cdw_sapp_branch` (`BRANCH_CODE`)
#   ON DELETE NO ACTION
#   ON UPDATE NO ACTION,
# ADD CONSTRAINT `CUST_SSN`
#   FOREIGN KEY (`CUST_SSN`)
#   REFERENCES `creditcard_capstone`.`cdw_sapp_customer` (`SSN`)
#   ON DELETE NO ACTION
#   ON UPDATE NO ACTION;